In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import auc
from sklearn.metrics import average_precision_score


In [2]:
file_paths = [
    "../data/cbb.csv",
    "../data/cbb13.csv",
    "../data/cbb14.csv",
    "../data/cbb15.csv",
    "../data/cbb16.csv",
    "../data/cbb17.csv",
    "../data/cbb18.csv",
    "../data/cbb19.csv",
    "../data/cbb20.csv",
    "../data/cbb21.csv",
    "../data/cbb22.csv",
    "../data/cbb23.csv",
    "../data/cbb24.csv",
]

# Loop through each dataset and print the first few rows
for info in file_paths:
        # Load the dataset
        data = pd.read_csv(info)
        
        # Print the dataset name and the first 5 rows
        print(f"Dataset: {info}")
        print(data.head())
        print("\n" + "="*50 + "\n")

Dataset: ../data/cbb.csv
             TEAM CONF   G   W  ADJOE  ADJDE  BARTHAG  EFG_O  EFG_D   TOR  \
0  North Carolina  ACC  40  33  123.3   94.9   0.9531   52.6   48.1  15.4   
1       Wisconsin  B10  40  36  129.1   93.6   0.9758   54.8   47.7  12.4   
2        Michigan  B10  40  33  114.4   90.4   0.9375   53.9   47.7  14.0   
3      Texas Tech  B12  38  31  115.2   85.2   0.9696   53.5   43.0  17.7   
4         Gonzaga  WCC  39  37  117.8   86.3   0.9728   56.6   41.1  16.2   

   ...  FTRD  2P_O  2P_D  3P_O  3P_D  ADJ_T   WAB  POSTSEASON  SEED  YEAR  
0  ...  30.4  53.9  44.6  32.7  36.2   71.7   8.6         2ND   1.0  2016  
1  ...  22.4  54.8  44.7  36.5  37.5   59.3  11.3         2ND   1.0  2015  
2  ...  30.0  54.7  46.8  35.2  33.2   65.9   6.9         2ND   3.0  2018  
3  ...  36.6  52.8  41.9  36.5  29.7   67.5   7.0         2ND   3.0  2019  
4  ...  26.9  56.3  40.0  38.2  29.0   71.5   7.7         2ND   1.0  2017  

[5 rows x 24 columns]


Dataset: ../data/cbb13.csv
    

In [4]:
# Initialize an empty list to store datasets
all_data = []

# Loop through each dataset and print the first few rows
for info in file_paths:
    # Load the dataset
    data = pd.read_csv(info)
    
    # Add a "YEAR" column if missing
    if "YEAR" not in data.columns:
        try:
            # Extract year from the file name
            year = int(info.split("/")[-1].replace("cbb", "").replace(".csv", ""))
            data["YEAR"] = year
        except ValueError:
            print(f"Unable to determine year for {info}")
            continue

    # Append the dataset to the list
    all_data.append(data)
    
    # Print the dataset name and the first 5 rows
    print(f"Dataset: {info}")
    print(data.head())
    print("\n" + "="*50 + "\n")

# Combine all datasets into a single DataFrame
combined_data = pd.concat(all_data, ignore_index=True)

# Print combined data info
print("Combined Dataset:")
print(combined_data.info())
print("\nFirst 5 rows of combined data:")
print(combined_data.head())


Dataset: ../data/cbb.csv
             TEAM CONF   G   W  ADJOE  ADJDE  BARTHAG  EFG_O  EFG_D   TOR  \
0  North Carolina  ACC  40  33  123.3   94.9   0.9531   52.6   48.1  15.4   
1       Wisconsin  B10  40  36  129.1   93.6   0.9758   54.8   47.7  12.4   
2        Michigan  B10  40  33  114.4   90.4   0.9375   53.9   47.7  14.0   
3      Texas Tech  B12  38  31  115.2   85.2   0.9696   53.5   43.0  17.7   
4         Gonzaga  WCC  39  37  117.8   86.3   0.9728   56.6   41.1  16.2   

   ...  FTRD  2P_O  2P_D  3P_O  3P_D  ADJ_T   WAB  POSTSEASON  SEED  YEAR  
0  ...  30.4  53.9  44.6  32.7  36.2   71.7   8.6         2ND   1.0  2016  
1  ...  22.4  54.8  44.7  36.5  37.5   59.3  11.3         2ND   1.0  2015  
2  ...  30.0  54.7  46.8  35.2  33.2   65.9   6.9         2ND   3.0  2018  
3  ...  36.6  52.8  41.9  36.5  29.7   67.5   7.0         2ND   3.0  2019  
4  ...  26.9  56.3  40.0  38.2  29.0   71.5   7.7         2ND   1.0  2017  

[5 rows x 24 columns]


Dataset: ../data/cbb13.csv
    

In [9]:
missing_values = combined_data.isnull().sum()
print("Missing values in each column:")
print(missing_values[missing_values > 0])

# Fill missing numerical values with column mean
numerical_columns = combined_data.select_dtypes(include=[np.number]).columns
combined_data[numerical_columns] = combined_data[numerical_columns].fillna(combined_data[numerical_columns].mean())


# Fill missing categorical values with the most common value
categorical_columns = combined_data.select_dtypes(include=['object', 'category']).columns
combined_data[categorical_columns] = combined_data[categorical_columns].fillna(combined_data[categorical_columns].mode().iloc[0])

# Verify missing values are handled
print("Missing Values After Handling:")
print(combined_data.isnull().sum())

Missing values in each column:
POSTSEASON    6401
dtype: int64
Missing Values After Handling:
TEAM          0
CONF          0
G             0
W             0
ADJOE         0
ADJDE         0
BARTHAG       0
EFG_O         0
EFG_D         0
TOR           0
TORD          0
ORB           0
DRB           0
FTR           0
FTRD          0
2P_O          0
2P_D          0
3P_O          0
3P_D          0
ADJ_T         0
WAB           0
POSTSEASON    0
SEED          0
YEAR          0
RK            0
EFGD_D        0
EFG%          0
EFGD%         0
dtype: int64


In [ ]:
# Check for duplicates
duplicates_count = combined_data.duplicated().sum()
print(f"Number of duplicate rows: {duplicates_count}")

Number of duplicate rows: 0
Rows after removing duplicates: 7761


In [14]:
# Convert categorical columns to 'category' type for optimization
categorical_columns = ["CONF", "POSTSEASON"]
for col in categorical_columns:
    if col in combined_data.columns:
        combined_data[col] = combined_data[col].astype("category")

# Verify data types
print(combined_data.dtypes)

TEAM            object
CONF          category
G                int64
W                int64
ADJOE          float64
ADJDE          float64
BARTHAG        float64
EFG_O          float64
EFG_D          float64
TOR            float64
TORD           float64
ORB            float64
DRB            float64
FTR            float64
FTRD           float64
2P_O           float64
2P_D           float64
3P_O           float64
3P_D           float64
ADJ_T          float64
WAB            float64
POSTSEASON    category
SEED           float64
YEAR             int64
RK             float64
EFGD_D         float64
EFG%           float64
EFGD%          float64
dtype: object
